In [1]:
from diviner import calib
from diviner import file_utils as fu
from diviner import plot_utils as pu
import pandas as pd

In [2]:
pd.get_option('display.max_columns')

20

In [3]:
pd.set_option('display.max_columns',200)

In [4]:
pump = fu.Div247DataPump('20110402')

In [5]:
df = pump.get_n_hours_from_t(3,10)

2011040210.div247
2011040211.div247
2011040212.div247


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 83120 entries, 2011-04-02 10:00:00.849000 to 2011-04-02 12:57:20.245999
Columns: 252 entries, sclk to is_calib
dtypes: bool(6), float64(241), int32(4), int64(1)
memory usage: 155.8 MB


In [7]:
calibrator = calib.Calibrator(df)

INFO:diviner.calib:Calibrating from 2011-04-02 10:00:00.849000 to 2011-04-02 12:57:20.245999.


In [8]:
calibrator.interpolate_bb_temps()

In [9]:
calibrator.calc_calib_times()

AttributeError: 'Calibrator' object has no attribute 'calc_calib_times'

In [12]:
calibrator.calibrate()

INFO:diviner.calib:Performing radiance correction on 2011-04-02 10:00:00.849000
/u/paige/maye/src/diviner/diviner/calib.py:847: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  ] *= self.norm_to_abs_converter.get_value(2, channel)


In [21]:
df = calibrator.df

In [22]:
bbviews_temps = df[df.is_bbview][['bb_1_temp_interp','bb_2_temp_interp']]

In [25]:
group = bbviews_temps.groupby(df.calib_block_labels)

In [26]:
group.agg?

Signature: group.agg(arg, *args, **kwargs)
Docstring:
Aggregate using one or more operations over the specified axis.



Parameters
----------
func : function, str, list or dict
    Function to use for aggregating the data. If a function, must either
    work when passed a DataFrame or when passed to DataFrame.apply.

    Accepted combinations are:

    - function
    - string function name
    - list of functions and/or function names, e.g. ``[np.sum, 'mean']``
    - dict of axis labels -> functions, function names or list of such.

*args
    Positional arguments to pass to `func`.
**kwargs
    Keyword arguments to pass to `func`.

Returns
-------
DataFrame, Series or scalar
    if DataFrame.agg is called with a single function, returns a Series
    if DataFrame.agg is called with several functions, returns a DataFrame
    if Series.agg is called with single function, returns a scalar
    if Series.agg is called with several functions, returns a Series


See Also
--------
pandas.DataF

In [24]:
bbtemps = bbviews_temps.groupby(df.calib_block_labels).agg(calibrator.skipped_mean,calibrator.BBV_NUM_SKIP_SAMPLE)

AttributeError: 'Calibrator' object has no attribute 'skipped_mean'

In [ ]:
calib.config.

In [ ]:
bbtemps

this is what calc_one_RBB gets for channel 9:

In [ ]:
calibrator.rbbtable.get_radiance(bbtemps.bb_2_temp_interp, 9)

convert all bb_2_temp_interp temperatures into radiances

In [ ]:
RBB_ch9_all = pd.Series(calibrator.rbbtable.get_radiance(df.bb_2_temp_interp, 9),index=df.index)

get only bbview data from these

In [ ]:
bbview_rbbs = RBB_ch9_all[df.is_bbview]

group them by calib blocks

In [ ]:
grouped = bbview_rbbs.groupby(df.calib_block_labels)

In [ ]:
d = dict(list(grouped))

In [ ]:
rbb_cb1 = d[1]

In [ ]:
calibrator.skipped_mean(rbb_cb1,16)

In [ ]:
for i,j in zip(calibrator.calc_one_RBB(return_values=True).a6_11.values,
               calibrator.calc_many_RBB(return_values=True).a6_11.values):
    print i-j

In [ ]:
%timeit calibrator.calc_one_RBB()

In [ ]:
%timeit calibrator.calc_many_RBB()

In [ ]:
calibrator.calc_many_RBB(return_values=True).b3_11.values[0]

In [ ]:
T_cols = ['bb_1_temp_interp','bb_2_temp_interp']

In [ ]:
bbviews_temps = df[df.is_bbview][T_cols]

In [ ]:
bbviews_temps

In [ ]:
grouped = bbviews_temps.groupby(df.calib_block_labels)

In [ ]:
grouped.size()

In [ ]:
bbtemps = grouped.agg(calibrator.skipped_mean, calibrator.BBV_NUM_SKIP_SAMPLE)

In [ ]:
bbtemps

In [ ]:
RBB = pd.DataFrame(index=calibrator.bbcal_times)

In [ ]:
calibrator.lookup_radiances_for_thermal_channels(bbtemps, RBB)

In [ ]:
RBB.b3_11

In [ ]:
bbtemps

In [ ]:
bbtemp = bbtemps['bb_2_temp_interp']

In [ ]:
bbtemp

In [ ]:
RBBs = calibrator.rbbtable.get_radiance(bbtemp,9)

In [ ]:
channel_rbbs = pd.Series(RBBs, index = bbtemps.